# Predicting label in train_prepared

### Notebook automatically generated from your model

Model Random forest, trained on 2020-06-25 21:30:50.

#### Generated on 2020-06-25 16:40:18.185656

prediction
This notebook will reproduce the steps for a BINARY_CLASSIFICATION on  train_prepared.
The main objective is to predict the variable label

#### Warning

The goal of this notebook is to provide an easily readable and explainable code that reproduces the main steps
of training the model. It is not complete: some of the preprocessing done by the DSS visual machine learning is not
replicated in this notebook. This notebook will not give the same results and model performance as the DSS visual machine
learning model.

Let's start with importing the required libs :

In [ ]:
import sys
import dataiku
import numpy as np
import pandas as pd
import sklearn as sk
import dataiku.core.pandasutils as pdu
from dataiku.doctor.preprocessing import PCA
from collections import defaultdict, Counter

And tune pandas display options:

In [ ]:
pd.set_option('display.width', 3000)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

#### Importing base data

The first step is to get our machine learning dataset:

In [ ]:
# We apply the preparation that you defined. You should not modify this.
preparation_steps = []
preparation_output_schema = {u'userModified': False, u'columns': [{u'type': u'string', u'name': u'url'}, {u'type': u'bigint', u'name': u'urlid'}, {u'type': u'string', u'name': u'boilerplate'}, {u'type': u'string', u'name': u'alchemy_category'}, {u'type': u'double', u'name': u'alchemy_category_score'}, {u'type': u'double', u'name': u'avglinksize'}, {u'type': u'double', u'name': u'commonlinkratio_1'}, {u'type': u'double', u'name': u'commonlinkratio_2'}, {u'type': u'double', u'name': u'commonlinkratio_3'}, {u'type': u'double', u'name': u'commonlinkratio_4'}, {u'type': u'double', u'name': u'compression_ratio'}, {u'type': u'double', u'name': u'embed_ratio'}, {u'type': u'bigint', u'name': u'framebased'}, {u'type': u'double', u'name': u'frameTagRatio'}, {u'type': u'bigint', u'name': u'hasDomainLink'}, {u'type': u'double', u'name': u'html_ratio'}, {u'type': u'double', u'name': u'image_ratio'}, {u'type': u'bigint', u'name': u'is_news'}, {u'type': u'bigint', u'name': u'lengthyLinkDomain'}, {u'type': u'bigint', u'name': u'linkwordscore'}, {u'type': u'bigint', u'name': u'news_front_page'}, {u'type': u'bigint', u'name': u'non_markup_alphanum_characters'}, {u'type': u'bigint', u'name': u'numberOfLinks'}, {u'type': u'bigint', u'name': u'numwords_in_url'}, {u'type': u'double', u'name': u'parametrizedLinkRatio'}, {u'type': u'double', u'name': u'spelling_errors_ratio'}, {u'type': u'bigint', u'name': u'label'}]}

ml_dataset_handle = dataiku.Dataset('train_prepared')
ml_dataset_handle.set_preparation_steps(preparation_steps, preparation_output_schema)
%time ml_dataset = ml_dataset_handle.get_dataframe(limit = 100000)

print ('Base data has %i rows and %i columns' % (ml_dataset.shape[0], ml_dataset.shape[1]))
# Five first records",
ml_dataset.head(5)

#### Initial data management

The preprocessing aims at making the dataset compatible with modeling.
At the end of this step, we will have a matrix of float numbers, with no missing values.
We'll use the features and the preprocessing steps defined in Models.

Let's only keep selected features

In [ ]:
ml_dataset = ml_dataset[[u'commonlinkratio_1', u'commonlinkratio_2', u'commonlinkratio_3', u'commonlinkratio_4', u'image_ratio', u'is_news', u'parametrizedLinkRatio', u'alchemy_category_score', u'lengthyLinkDomain', u'label', u'numwords_in_url', u'alchemy_category', u'boilerplate', u'spelling_errors_ratio', u'linkwordscore', u'non_markup_alphanum_characters', u'news_front_page', u'html_ratio', u'hasDomainLink', u'embed_ratio', u'frameTagRatio', u'avglinksize', u'numberOfLinks', u'compression_ratio']]

Let's first coerce categorical columns into unicode, numerical features into floats.

In [ ]:
# astype('unicode') does not work as expected

def coerce_to_unicode(x):
    if sys.version_info < (3, 0):
        if isinstance(x, str):
            return unicode(x,'utf-8')
        else:
            return unicode(x)
    else:
        return str(x)


categorical_features = [u'alchemy_category', u'boilerplate']
numerical_features = [u'commonlinkratio_1', u'commonlinkratio_2', u'commonlinkratio_3', u'commonlinkratio_4', u'image_ratio', u'is_news', u'parametrizedLinkRatio', u'alchemy_category_score', u'lengthyLinkDomain', u'numwords_in_url', u'spelling_errors_ratio', u'linkwordscore', u'non_markup_alphanum_characters', u'news_front_page', u'html_ratio', u'hasDomainLink', u'embed_ratio', u'frameTagRatio', u'avglinksize', u'numberOfLinks', u'compression_ratio']
text_features = []
from dataiku.doctor.utils import datetime_to_epoch
for feature in categorical_features:
    ml_dataset[feature] = ml_dataset[feature].apply(coerce_to_unicode)
for feature in text_features:
    ml_dataset[feature] = ml_dataset[feature].apply(coerce_to_unicode)
for feature in numerical_features:
    if ml_dataset[feature].dtype == np.dtype('M8[ns]'):
        ml_dataset[feature] = datetime_to_epoch(ml_dataset[feature])
    else:
        ml_dataset[feature] = ml_dataset[feature].astype('double')

We are now going to handle the target variable and store it in a new variable:

In [ ]:
target_map = {u'1': 1, u'0': 0}
ml_dataset['__target__'] = ml_dataset['label'].map(str).map(target_map)
del ml_dataset['label']


# Remove rows for which the target is unknown.
ml_dataset = ml_dataset[~ml_dataset['__target__'].isnull()]

#### Cross-validation strategy

The dataset needs to be split into 2 new sets, one that will be used for training the model (train set)
and another that will be used to test its generalization capability (test set)

This is a simple cross-validation strategy.

In [ ]:
train, test = pdu.split_train_valid(ml_dataset, prop=0.8)
print ('Train data has %i rows and %i columns' % (train.shape[0], train.shape[1]))
print ('Test data has %i rows and %i columns' % (test.shape[0], test.shape[1]))

#### Features preprocessing

The first thing to do at the features level is to handle the missing values.
Let's reuse the settings defined in the model

In [ ]:
drop_rows_when_missing = []
impute_when_missing = [{'impute_with': u'MEAN', 'feature': u'commonlinkratio_1'}, {'impute_with': u'MEAN', 'feature': u'commonlinkratio_2'}, {'impute_with': u'MEAN', 'feature': u'commonlinkratio_3'}, {'impute_with': u'MEAN', 'feature': u'commonlinkratio_4'}, {'impute_with': u'MEAN', 'feature': u'image_ratio'}, {'impute_with': u'MEAN', 'feature': u'is_news'}, {'impute_with': u'MEAN', 'feature': u'parametrizedLinkRatio'}, {'impute_with': u'MEAN', 'feature': u'alchemy_category_score'}, {'impute_with': u'MEAN', 'feature': u'lengthyLinkDomain'}, {'impute_with': u'MEAN', 'feature': u'numwords_in_url'}, {'impute_with': u'MEAN', 'feature': u'spelling_errors_ratio'}, {'impute_with': u'MEAN', 'feature': u'linkwordscore'}, {'impute_with': u'MEAN', 'feature': u'non_markup_alphanum_characters'}, {'impute_with': u'MEAN', 'feature': u'news_front_page'}, {'impute_with': u'MEAN', 'feature': u'html_ratio'}, {'impute_with': u'MEAN', 'feature': u'hasDomainLink'}, {'impute_with': u'MEAN', 'feature': u'embed_ratio'}, {'impute_with': u'MEAN', 'feature': u'frameTagRatio'}, {'impute_with': u'MEAN', 'feature': u'avglinksize'}, {'impute_with': u'MEAN', 'feature': u'numberOfLinks'}, {'impute_with': u'MEAN', 'feature': u'compression_ratio'}]

# Features for which we drop rows with missing values"
for feature in drop_rows_when_missing:
    train = train[train[feature].notnull()]
    test = test[test[feature].notnull()]
    print ('Dropped missing records in %s' % feature)

# Features for which we impute missing values"
for feature in impute_when_missing:
    if feature['impute_with'] == 'MEAN':
        v = train[feature['feature']].mean()
    elif feature['impute_with'] == 'MEDIAN':
        v = train[feature['feature']].median()
    elif feature['impute_with'] == 'CREATE_CATEGORY':
        v = 'NULL_CATEGORY'
    elif feature['impute_with'] == 'MODE':
        v = train[feature['feature']].value_counts().index[0]
    elif feature['impute_with'] == 'CONSTANT':
        v = feature['value']
    train[feature['feature']] = train[feature['feature']].fillna(v)
    test[feature['feature']] = test[feature['feature']].fillna(v)
    print ('Imputed missing values in feature %s with value %s' % (feature['feature'], coerce_to_unicode(v)))

We can now handle the categorical features (still using the settings defined in Models):

Let's dummy-encode the following features.
A binary column is created for each of the 100 most frequent values.

In [ ]:
LIMIT_DUMMIES = 100

categorical_to_dummy_encode = [u'alchemy_category', u'boilerplate']

# Only keep the top 100 values
def select_dummy_values(train, features):
    dummy_values = {}
    for feature in categorical_to_dummy_encode:
        values = [
            value
            for (value, _) in Counter(train[feature]).most_common(LIMIT_DUMMIES)
        ]
        dummy_values[feature] = values
    return dummy_values

DUMMY_VALUES = select_dummy_values(train, categorical_to_dummy_encode)

def dummy_encode_dataframe(df):
    for (feature, dummy_values) in DUMMY_VALUES.items():
        for dummy_value in dummy_values:
            dummy_name = u'%s_value_%s' % (feature, coerce_to_unicode(dummy_value))
            df[dummy_name] = (df[feature] == dummy_value).astype(float)
        del df[feature]
        print ('Dummy-encoded feature %s' % feature)

dummy_encode_dataframe(train)

dummy_encode_dataframe(test)

Let's rescale numerical features

In [ ]:
rescale_features = {u'parametrizedLinkRatio': u'AVGSTD', u'alchemy_category_score': u'AVGSTD', u'linkwordscore': u'AVGSTD', u'embed_ratio': u'AVGSTD', u'spelling_errors_ratio': u'AVGSTD', u'hasDomainLink': u'AVGSTD', u'commonlinkratio_1': u'AVGSTD', u'commonlinkratio_2': u'AVGSTD', u'commonlinkratio_3': u'AVGSTD', u'commonlinkratio_4': u'AVGSTD', u'numberOfLinks': u'AVGSTD', u'compression_ratio': u'AVGSTD', u'avglinksize': u'AVGSTD', u'numwords_in_url': u'AVGSTD', u'lengthyLinkDomain': u'AVGSTD', u'image_ratio': u'AVGSTD', u'is_news': u'AVGSTD', u'news_front_page': u'AVGSTD', u'non_markup_alphanum_characters': u'AVGSTD', u'frameTagRatio': u'AVGSTD', u'html_ratio': u'AVGSTD'}
for (feature_name, rescale_method) in rescale_features.items():
    if rescale_method == 'MINMAX':
        _min = train[feature_name].min()
        _max = train[feature_name].max()
        scale = _max - _min
        shift = _min
    else:
        shift = train[feature_name].mean()
        scale = train[feature_name].std()
    if scale == 0.:
        del train[feature_name]
        del test[feature_name]
        print ('Feature %s was dropped because it has no variance' % feature_name)
    else:
        print ('Rescaled %s' % feature_name)
        train[feature_name] = (train[feature_name] - shift).astype(np.float64) / scale
        test[feature_name] = (test[feature_name] - shift).astype(np.float64) / scale

#### Modeling

Before actually creating our model, we need to split the datasets into their features and labels parts:

In [ ]:
train_X = train.drop('__target__', axis=1)
test_X = test.drop('__target__', axis=1)

train_Y = np.array(train['__target__'])
test_Y = np.array(test['__target__'])

Now we can finally create our model !

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100,
    random_state=1337,
    max_depth=15,
    min_samples_leaf=1,
    verbose=2)

... And train it

In [ ]:
%time clf.fit(train_X, train_Y)

Build up our result dataset

The model is now trained, we can apply it to our test set:

In [ ]:
%time _predictions = clf.predict(test_X)
%time _probas = clf.predict_proba(test_X)
predictions = pd.Series(data=_predictions, index=test_X.index, name='predicted_value')
cols = [
    u'probability_of_value_%s' % label
    for (_, label) in sorted([(int(target_map[label]), label) for label in target_map])
]
probabilities = pd.DataFrame(data=_probas, index=test_X.index, columns=cols)

# Build scored dataset
results_test = test_X.join(predictions, how='left')
results_test = results_test.join(probabilities, how='left')
results_test = results_test.join(test['__target__'], how='left')
results_test = results_test.rename(columns= {'__target__': 'label'})

Let's have a look at feature importances

In [ ]:
feature_importances_data = []
features = train_X.columns
for feature_name, feature_importance in zip(features, clf.feature_importances_):
    feature_importances_data.append({
        'feature': feature_name,
        'importance': feature_importance
    })

# Plot the results
pd.DataFrame(feature_importances_data)\
    .set_index('feature')\
    .sort_values(by='importance')[-10::]\
    .plot(title='Top 10 most important variables',
          kind='barh',
          figsize=(10, 6),
          color='#348ABD',
          alpha=0.6,
          lw='1',
          edgecolor='#348ABD',
          grid=False,)

#### Results

You can measure the model's accuracy:

In [ ]:
from dataiku.doctor.utils.metrics import mroc_auc_score
test_Y_ser = pd.Series(test_Y)
print ('AUC value:', mroc_auc_score(test_Y_ser, _probas))

We can also view the predictions directly.
Since scikit-learn only predicts numericals, the labels have been mapped to 0,1,2 ...
We need to 'reverse' the mapping to display the initial labels.

In [ ]:
inv_map = { target_map[label] : label for label in target_map}
predictions.map(inv_map)

That's it. It's now up to you to tune your preprocessing, your algo, and your analysis !
